In [7]:
import numpy as np
import matplotlib.pyplot as plt


# Propogate Divergence & Waist Rays of Gaussian Beamlets - NONastigmatic beamlets

# System Parameters
wl     = 2.2e-6  # wavelength
d      = 5e3        # propagation distance
f      = 1e50     # focal length
sample = 1024       # number of detector pixels across
amax   = 5e-1     # maximum aperture radius
amin   = 0      # minimum aperture radius
OF     = 2      # Beamlet Overlap Factor
numB   = 2     # number of beams across the grid

# Calculate beam waist
wo = 2*amax*OF/(2*numB)
Zr = (np.pi*wo**2)/wl # Rayleigh range
k  = 2*np.pi/wl

print(amax**2 / (d*wl),'Fresnel Number')
print(wo,'Beam Waist')
print(wo/wl,'waves')

22.727272727272727 Fresnel Number
0.5 Beam Waist
227272.72727272726 waves


In [8]:
# Predefine empty array for ray distribution - gonna put x & y components of rays here
x = np.linspace(-amax,amax,numB)
y = np.linspace(-amax,amax,numB)
x,y = np.meshgrid(x,y)

# Configure system ABCD Matrix -> these become optical elements in future revisions
focus = np.array([[1,0],
                  [-1/f,1]])

propg = np.array([[1,d],
                  [0,1]])

# Calculate system ABCD Matrix
system = np.matmul(propg,focus)

# Disperse q parameter across the beamlets
qmat = (1j*Zr) # np.ones((numB,numB))*(1j*Zr) # stores initial complex curvature for all beamlets z=0

In [9]:
# Define waist ray - unit height
waist = np.array([[1]
                 ,[0]]) # top element is y, bottom element is slope

# Create some empty boxes to throw our waist vectors into - this locates the beamlet
pwx = np.zeros([numB,numB],dtype=object)
pwy = np.zeros([numB,numB],dtype=object)

# Create x & y arrays of waist & divergence rays
for ijk in range(numB):
    for lmn in range(numB):
        # Generate scale matricies
        scalex = np.array([[x[ijk,lmn],0],[0,1]])
        scaley = np.array([[y[ijk,lmn],0],[0,1]])
        
        # Place our ray vectors in a box
        pwx[ijk,lmn] = waist*(x[ijk,lmn]) # creates the waist ray
        pwy[ijk,lmn] = waist*(y[ijk,lmn])

In [10]:
# Propagate waist rays
for abc in range(numB):
    for xyz in range(numB):
        pwx[abc,xyz] = np.matmul(system,pwx[abc,xyz])
        pwy[abc,xyz] = np.matmul(system,pwy[abc,xyz])

# Propagate q parameter
qprop = (system[0,0]*qmat + system[0,1])/(system[1,0]*qmat + system[1,1])

In [11]:
# Now we have a qparam and a grid of waist rays, lets-a plot some-a beamses

# Define Sample Plane
u = np.linspace(-2*W,2*W,sample)
v = np.linspace(-2*W,2*W,sample)
u,v = np.meshgrid(u,v)
afield = np.zeros([sample,sample]) # define analysis box

for px in range(numB):
    for py in range(numB):
        
        # Grab analysis x & y comps from waist for beam location
        imguv_parsx = imgwx[px,py] 
        imguv_parsy = imgwy[px,py]
        
        if np.sqrt(x[px,py]**2 + y[px,py]**2) <= amax and np.sqrt(x[px,py]**2 + y[px,py]**2) >= amin:
            p = np.square(u-imguv_parsx[0,0]) + np.square(v-imguv_parsy[0,0]) # define squared radial coord
            realexp = np.exp(-p/(wp[px,py])**2)
            imagexp = np.exp(-1j*(k*tbox[px,py] + k*(p/(2*Rbox[px,py]))-np.arctan(tbox[px,py]/Zr)))
            amptude = wo/wp[px,py]
            afield = afield + amptude*realexp*imagexp 
        

NameError: name 'W' is not defined